In [ ]:
import os
from PIL import Image
from ipywidgets import Text, Button, Output, VBox
from IPython.display import display

def process_images(folder_path, save_directory, output):
    """
    Scans a folder, rotates portrait images to landscape, and splits each image
    into a square and rectangle version, saving them to a specified directory.

    Args:
        folder_path (str): The path to the folder containing the images.
        save_directory (str): The path to the directory where processed images will be saved.
        output (Output): The output widget to display messages.
    """
    try:
        # Check if folder exists
        if not os.path.isdir(folder_path):
            output.append_stdout(f"❌ Error: Folder not found at '{folder_path}'\n")
            return

        # Create save directory if it doesn't exist
        os.makedirs(save_directory, exist_ok=True)

        output.append_stdout(f"📂 Processing images in: '{folder_path}'\n")
        output.append_stdout(f"💾 Saving processed images to: '{save_directory}'\n")

        # Process each file
        for filename in os.listdir(folder_path):
            filepath = os.path.join(folder_path, filename)

            if os.path.isfile(filepath) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                try:
                    img = Image.open(filepath)
                    width, height = img.size

                    output.append_stdout(f"🔵 Processing: {filename} (Width: {width}, Height: {height})\n")

                    # Rotate portrait images
                    if height > width:
                        img = img.rotate(90, expand=True)
                        output.append_stdout("  ↩️ Rotated to landscape.\n")
                        width, height = img.size

                    # Create square crop (centered)
                    min_dim = min(width, height)
                    left = (width - min_dim) // 2
                    top = (height - min_dim) // 2
                    right = left + min_dim
                    bottom = top + min_dim
                    square_img = img.crop((left, top, right, bottom))

                    # Save square image
                    square_filename = f"{os.path.splitext(filename)[0]}_square{os.path.splitext(filename)[1]}"
                    square_filepath = os.path.join(save_directory, square_filename)
                    square_img.save(square_filepath)
                    output.append_stdout(f"  🟦 Saved square image: {square_filename}\n")

                    # Save rectangle (original rotated image)
                    rectangle_filename = f"{os.path.splitext(filename)[0]}_rectangle{os.path.splitext(filename)[1]}"
                    rectangle_filepath = os.path.join(save_directory, rectangle_filename)
                    img.save(rectangle_filepath)
                    output.append_stdout(f"  🟥 Saved rectangle image: {rectangle_filename}\n")

                    img.close()

                except Exception as e:
                    output.append_stdout(f"⚠️ Error processing {filename}: {e}\n")

        output.append_stdout("✅ Image processing complete.\n")

    except Exception as e:
        output.append_stdout(f"🔥 Unexpected error: {e}\n")


def image_processing_app():
    """
    Creates and displays the interactive widgets for the image processing application.
    """
    folder_path_text = Text(description="Folder Path:", value="C:/your/folder/here")
    save_directory_text = Text(description="Save Directory:", value="C:/your/save/folder")
    process_button = Button(description="Process Images")
    output = Output()

    def on_process_button_click(button):
        # Clear old output
        output.clear_output()
        # Start processing
        process_images(folder_path_text.value, save_directory_text.value, output)

    process_button.on_click(on_process_button_click)

    # Layout
    app_layout = VBox([folder_path_text, save_directory_text, process_button, output])
    display(app_layout)


# Only run this if inside Jupyter Notebook
try:
    get_ipython()
    image_processing_app()
except NameError:
    print("This script is designed to be run inside a Jupyter Notebook environment.")
